In [ ]:
!wget https://datasets.simula.no/downloads/kvasir-seg.zip

!unzip kvasir-seg.zip

--2025-05-28 21:37:34--  https://datasets.simula.no/downloads/kvasir-seg.zip
Resolving datasets.simula.no (datasets.simula.no)... 128.39.36.14
Connecting to datasets.simula.no (datasets.simula.no)|128.39.36.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46227172 (44M) [application/zip]
Saving to: ‘kvasir-seg.zip’

kvasir-seg.zip      100%[===================>]  44.08M  18.4MB/s    in 2.4s    

2025-05-28 21:37:38 (18.4 MB/s) - ‘kvasir-seg.zip’ saved [46227172/46227172]

Archive:  kvasir-seg.zip
   creating: Kvasir-SEG/
  inflating: Kvasir-SEG/kavsir_bboxes.json  
   creating: Kvasir-SEG/images/
  inflating: Kvasir-SEG/images/ck2bxiswtxuw80838qkisqjwz.jpg  
  inflating: Kvasir-SEG/images/ck2bxknhjvs1x0794iogrq49k.jpg  
  inflating: Kvasir-SEG/images/ck2bxlujamu330725szlc2jdu.jpg  
  inflating: Kvasir-SEG/images/ck2bxpfgxu2mk0748gsh7xelu.jpg  
  inflating: Kvasir-SEG/images/ck2bxqz3evvg20794iiyv5v2m.jpg  
  inflating: Kvasir-SEG/images/ck2bxskgxxzfv08386xkqt

In [ ]:
import torch
import torch.nn as nn
import timm
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn.functional as F # Import functional for interpolation

class MDCB(nn.Module):
    # Multidilation Convolutional Block for multi-scale feature extraction
    def __init__(self, in_ch, out_ch, dilations=[1, 2, 4]):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=d, dilation=d)
            for d in dilations
        ])
        self.relu = nn.ReLU(inplace=True)
        self.fuse = nn.Conv2d(len(dilations)*out_ch, out_ch, kernel_size=1)

    def forward(self, x):
        features = [conv(x) for conv in self.convs]
        x = torch.cat(features, dim=1)
        x = self.fuse(x)
        return self.relu(x)

class MFAB(nn.Module):
    # Multifeature Aggregation Block, similar to U-Net skip connection
    def __init__(self, in_chs, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(sum(in_chs), out_ch, kernel_size=1)

    def forward(self, *features):
        # Before concatenation, ensure spatial dimensions match
        # Assuming features[0] has the highest resolution among the inputs to MFAB
        target_size = features[0].shape[2:] # Get H, W from the first feature

        resized_features = [features[0]] # Keep the first feature as is

        # Resize subsequent features to match the target_size
        for i in range(1, len(features)):
            # Use bicubic interpolation for better quality, align_corners=False is standard
            resized_feature = F.interpolate(features[i], size=target_size, mode='bicubic', align_corners=False)
            resized_features.append(resized_feature)

        # Now concatenate the resized features
        x = torch.cat(resized_features, dim=1)
        return self.conv(x)

class SwinENet(nn.Module):
    def __init__(self, num_classes=1, backbone='swin_base_patch4_window7_224'):
        super().__init__()
        self.encoder = timm.create_model(backbone, pretrained=True, features_only=True)
        encoder_channels = self.encoder.feature_info.channels()
        print("Encoder channels:", encoder_channels) # Print encoder channels
        self.mdcb = nn.ModuleList([MDCB(ch, ch) for ch in encoder_channels])
        # The decoder upsamples. Need to ensure sizes match for concatenation later
        self.decoder = nn.ModuleList([
            # Adjust deconv input channel based on MDCB output of the deepest feature
            nn.ConvTranspose2d(encoder_channels[-1], 256, 2, stride=2),
            # The input to the next deconv should be the sum of deconv output and potentially a skip connection
            # However, the current design concatenates first 3 encoder features in MFAB *after* decoding.
            # This seems unusual for a typical U-Net structure. Let's stick to the original structure's intent
            # and assume the decoder just upsamples the deepest feature.
            nn.ConvTranspose2d(256, 128, 2, stride=2),
            nn.ConvTranspose2d(128, 64, 2, stride=2)
        ])
        # Adjust MFAB input channels based on expected decoder output and skip connections
        # The skip connections likely come from the MDCB outputs corresponding to the decoder stages
        # The original MFAB expected [64, 128, 256] from feats[0], feats[1], feats[2]
        # Let's verify the output channels of MDCB corresponding to these feats
        mdcb_output_channels_for_mfab = [encoder_channels[0], encoder_channels[1], encoder_channels[2]]
        # Ensure MFAB channels match the actual output channels of the MDCBs used for skip connections.
        # Since MDCB's out_ch is set to match in_ch, this should match the encoder_channels.
        self.mfab = MFAB(mdcb_output_channels_for_mfab, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        feats = self.encoder(x)  # list of features from Swin backbone

        # Check if features are in NHWC and permute if necessary
        processed_feats = []
        print("\nOriginal Feature map shapes from encoder:")
        for i, f in enumerate(feats):
            print(f"  Stage {i}: {f.shape}")
            # Check if channel dim (dim 1) is significantly smaller than spatial dims (dim 2, 3)
            # A more robust check might involve looking at feature_info's output shape expectations
            # but the traceback strongly suggests NHWC output for some stages.
            # Let's assume if it's 4D and channels are the last dim based on the traceback [B, H, W, C]
            if f.ndim == 4 and f.shape[-1] == self.encoder.feature_info.channels()[i]:
                 print(f"Permuting feature map {i} from NHWC {f.shape} to NCHW")
                 f = f.permute(0, 3, 1, 2)
            processed_feats.append(f)
        feats = processed_feats


        # Print shapes of feature maps *after* potential permutation
        print("\nFeature map shapes from encoder (after potential permute to NCHW):")
        for i, f in enumerate(feats):
            print(f"  Stage {i}: {f.shape}")

        # Now, apply MDCB. The input channels for MDCB were initialized based on encoder_channels.
        # We need to verify if the *actual* channels of the permuted features match the expected channels.
        print("\nChecking channel match for MDCB inputs:")
        for i, (mdcb_module, feature_map) in enumerate(zip(self.mdcb, feats)):
             expected_in_channels = mdcb_module.convs[0].in_channels # Get expected input channels from the first conv in MDCB
             actual_in_channels = feature_map.shape[1]
             print(f"  MDCB {i}: Expected {expected_in_channels}, Actual {actual_in_channels}")
             if expected_in_channels != actual_in_channels:
                  # This indicates feature_info.channels() does not match actual output channels
                  # If this happens, MDCB initialization needs to be fixed based on actual shapes
                  raise ValueError(f"Channel mismatch for MDCB at index {i} AFTER PERMUTE: Expected {expected_in_channels}, got {actual_in_channels}. Feature map shape: {feature_map.shape}")


        # Apply MDCB to each feature map
        feats = [mdcb(f) for mdcb, f in zip(self.mdcb, feats)]

        # Decoder path - Upsample the deepest feature
        x = feats[-1] # Start decoding from the deepest feature

        # Print shape before decoding
        print(f"\nShape before decoding: {x.shape}")
        for i, dec in enumerate(self.decoder):
            x = dec(x)
            print(f"Shape after decoder layer {i}: {x.shape}")


        # MFAB usage - concatenates first three *processed* features (after MDCB)
        # These features will now be resized within the MFAB's forward method
        mfab_input_features = [feats[0], feats[1], feats[2]]

        # Print shapes of features going into MFAB (after MDCB)
        print("\nFeature shapes going into MFAB (after MDCB):")
        for i, f in enumerate(mfab_input_features):
            print(f"  MFAB input {i}: {f.shape}")

        # Check if MFAB input channels match its initialization
        # The lines below were causing the error and are for debugging/verification
        # expected_mfab_in_channels = sum(self.mfab.conv.in_channels for _ in mfab_input_features)
        # expected_mfab_in_channels = self.mfab.conv.in_channels
        # expected_mfab_input_channels_list = [c.conv.in_channels for c in self.mfab.convs] # This line caused the error

        # Let's check the sum of actual channels of the features passed to MFAB
        actual_mfab_in_channels_sum = sum([f.shape[1] for f in mfab_input_features])

        # If MFAB was initialized correctly based on `mdcb_output_channels_for_mfab` (which came from `encoder_channels`)
        # and MDCBs preserved channels, then `actual_mfab_in_channels_sum` should equal `self.mfab.conv.in_channels`.
        print(f"\nChecking channel match for MFAB input: Expected sum {self.mfab.conv.in_channels}, Actual sum {actual_mfab_in_channels_sum}")
        if self.mfab.conv.in_channels != actual_mfab_in_channels_sum:
             # This suggests the MFAB initialization was incorrect based on the actual processed feature channels
             # If this happens, the MFAB initialization `mfab_output_channels_for_mfab = [...]` needs correction.
             # It should sum the *actual* channel counts of feats[0], feats[1], feats[2] *after* MDCB.
             # Since MDCB(ch, ch) preserves channels, the original initialization should be correct if encoder_channels is correct.
             raise ValueError(f"Channel mismatch for MFAB input: Expected sum {self.mfab.conv.in_channels}, got sum {actual_mfab_in_channels_sum}. This likely indicates an issue with MDCB not preserving channels or incorrect encoder_channels used for MFAB initialization.")


        x = self.mfab(*mfab_input_features) # Pass the first three *processed* features

        # Print shape after MFAB
        print(f"Shape after MFAB: {x.shape}")

        x = self.final_conv(x)

        # Print shape after final conv
        print(f"Shape after final conv: {x.shape}")


        # Upsample the final output to the original input size (224x224) if needed.
        # Assuming the goal is to output a segmentation map of the same size as the input image.
        # The decoder brings the deepest feature up to a certain resolution.
        # We need to check the final output shape and upsample if necessary.
        # Based on the decoder layers, the output of the last deconv (64 channels) will be
        # 224 / 32 * 2 * 2 * 2 = 56 * 8 = 448... No, this calculation is wrong.
        # Starting from the last encoder stage, let's say it's 7x7 for 224x224 input patch size 32.
        # Deconv 1: 7x7 -> 14x14 (stride 2) -> 256 channels
        # Deconv 2: 14x14 -> 28x28 (stride 2) -> 128 channels
        # Deconv 3: 28x28 -> 56x56 (stride 2) -> 64 channels
        # The MFAB input features are likely from stages with spatial sizes like 56x56, 28x28, 14x14.
        # The MFAB concatenates these and outputs 64 channels at the highest resolution (56x56).
        # The final_conv changes channels to 1. The output shape before final upsample would be [B, 1, 56, 56].
        # Need to upsample to [B, 1, 224, 224].
        if x.shape[2:] != (224, 224):
             print(f"Final output shape {x.shape[2:]} is not 224x224. Resizing.")
             x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False) # Use bilinear for segmentation output

        return x

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

class KvasirSegDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None, mask_transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.mask_transform = mask_transform
        self.images = sorted(os.listdir(images_dir))
        self.mask_extension = '.jpg'  # Define the mask file extension here


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.images_dir, img_name)
        base_name = os.path.splitext(img_name)[0] # Get filename without extension
        mask_name = base_name + self.mask_extension
        mask_path = os.path.join(self.masks_dir, mask_name)

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale mask

        if self.transform:
            image = self.transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        else:
            mask = T.ToTensor()(mask)
            # For binary segmentation, ensure mask is 0/1
            mask = (mask > 0).float()
        return image, mask

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T

image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

mask_transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

dataset = KvasirSegDataset(
    images_dir='/content/Kvasir-SEG/images',
    masks_dir='/content/Kvasir-SEG/masks',
    transform=image_transform,
    mask_transform=mask_transform
)

loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
import torch

model = SwinENet(num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()  # or Dice Loss

for images, masks in loader:
    preds = model(images)
    loss = criterion(preds, masks)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Encoder channels: [128, 256, 512, 1024]

Original Feature map shapes from encoder:
  Stage 0: torch.Size([8, 56, 56, 128])
Permuting feature map 0 from NHWC torch.Size([8, 56, 56, 128]) to NCHW
  Stage 1: torch.Size([8, 28, 28, 256])
Permuting feature map 1 from NHWC torch.Size([8, 28, 28, 256]) to NCHW
  Stage 2: torch.Size([8, 14, 14, 512])
Permuting feature map 2 from NHWC torch.Size([8, 14, 14, 512]) to NCHW
  Stage 3: torch.Size([8, 7, 7, 1024])
Permuting feature map 3 from NHWC torch.Size([8, 7, 7, 1024]) to NCHW

Feature map shapes from encoder (after potential permute to NCHW):
  Stage 0: torch.Size([8, 128, 56, 56])
  Stage 1: torch.Size([8, 256, 28, 28])
  Stage 2: torch.Size([8, 512, 14, 14])
  Stage 3: torch.Size([8, 1024, 7, 7])

Checking channel match for MDCB inputs:
  MDCB 0: Expected 128, Actual 128
  MDCB 1: Expected 256, Actual 256
  MDCB 2: Expected 512, Actual 512
  MDCB 3: Expected 1024, Actual 1024

Shape before decoding: torch.Size([8, 1024, 7, 7])
Shape after d

In [ ]:
# Save the trained model
model_save_path = '/content/swinenet_kvasir_seg.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# --- Code to save prediction masks for all examples ---
model.eval() # Set model to evaluation mode
output_masks_dir = '/content/predicted_masks'
os.makedirs(output_masks_dir, exist_ok=True) # Create directory if it doesn't exist

# Removed num_examples_to_save and related batch calculation
# The code will now iterate through all batches in the loader

# Initialize a counter for the total number of masks saved
total_saved_count = 0

with torch.no_grad(): # Disable gradient calculation for inference
    # Iterate through ALL batches in the loader
    for i, (images, masks) in enumerate(loader):
        preds = model(images)

        # Apply sigmoid and threshold to get binary masks (assuming BCEWithLogitsLoss was used)
        predicted_masks = torch.sigmoid(preds) > 0.5

        # Iterate over each example in the current batch
        for j in range(images.size(0)):
            # Convert the predicted mask to a PIL Image
            # Squeeze to remove batch and channel dimensions, convert to uint8
            mask_img = predicted_masks[j].squeeze(0).byte().cpu() * 255 # Convert to 0-255 for saving
            mask_pil = Image.fromarray(mask_img.numpy(), mode='L') # 'L' for grayscale

            # Save the predicted mask using the global counter for filename index
            mask_filename = f"predicted_mask_{total_saved_count:04d}.png" # e.g., predicted_mask_0000.png
            mask_filepath = os.path.join(output_masks_dir, mask_filename)
            mask_pil.save(mask_filepath)
            print(f"Saved predicted mask to {mask_filepath}")

            # Increment the total saved count
            total_saved_count += 1

print(f"Finished saving {total_saved_count} predicted masks.")

Streaming output truncated to the last 5000 lines.
Saved predicted mask to /content/predicted_masks/predicted_mask_0108.png
Saved predicted mask to /content/predicted_masks/predicted_mask_0109.png
Saved predicted mask to /content/predicted_masks/predicted_mask_0110.png
Saved predicted mask to /content/predicted_masks/predicted_mask_0111.png

Original Feature map shapes from encoder:
  Stage 0: torch.Size([8, 56, 56, 128])
Permuting feature map 0 from NHWC torch.Size([8, 56, 56, 128]) to NCHW
  Stage 1: torch.Size([8, 28, 28, 256])
Permuting feature map 1 from NHWC torch.Size([8, 28, 28, 256]) to NCHW
  Stage 2: torch.Size([8, 14, 14, 512])
Permuting feature map 2 from NHWC torch.Size([8, 14, 14, 512]) to NCHW
  Stage 3: torch.Size([8, 7, 7, 1024])
Permuting feature map 3 from NHWC torch.Size([8, 7, 7, 1024]) to NCHW

Feature map shapes from encoder (after potential permute to NCHW):
  Stage 0: torch.Size([8, 128, 56, 56])
  Stage 1: torch.Size([8, 256, 28, 28])
  Stage 2: torch.Size([8

In [ ]:
!pip install --upgrade torchmetrics

In [ ]:
# Install torchmetrics if not already installed, and upgrade to ensure latest version
# Re-run this command to ensure torchmetrics is up-to-date


import torch
import torchmetrics
# Import Dice specifically from torchmetrics
from torchmetrics.segmentation import DiceScore # Import Dice directly from torchmetrics
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
import timm
import torch.nn as nn
import torchmetrics # Keep this if you use torchmetrics.JaccardIndex etc. directly

# Re-define model and classes if running in a fresh environment, otherwise skip
# Assuming the model definition, dataset, and loader are already defined as before

# Load the trained model state dictionary
# The model definition and loader instantiation should be run *before* this cell
# if you are restarting the notebook or running cells out of order.
# For a sequential run, they are already defined in previous cells.
try:
    model # Check if model exists
except NameError:
    # If model is not defined (e.g., running this cell first), instantiate it
    print("Model not found, instantiating SwinENet.")
    # Assuming SwinENet class is defined in a previous cell or imported
    # If it's not defined yet, uncomment and potentially copy its definition here
    # from your model definition cell:
    # class SwinENet(...): ...
    model = SwinENet(num_classes=1)

model_save_path = '/content/swinenet_kvasir_seg.pth'
if os.path.exists(model_save_path):
    try:
        # Ensure the model is on the correct device before loading state_dict if using GPU
        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # model.to(device)
        model.load_state_dict(torch.load(model_save_path))
        print(f"Loaded model from {model_save_path}")
    except RuntimeError as e:
        print(f"Error loading model state_dict: {e}")
        print("This might happen if the model architecture definition has changed or device mismatch.")
        # Handle error, e.g., continue without loading or raise the exception
        # raise e # Uncomment to stop execution on load error
else:
    print(f"Model state dict not found at {model_save_path}. Cannot evaluate.")
    # Decide how to proceed: train a new model, skip evaluation, etc.
    # For this example, we'll print a warning and continue (model will be untrained)

model.eval() # Set model to evaluation mode

# Define the metrics to calculate
# Use 'binary' for binary segmentation (1 class + background)
# torchmetrics.Dice is the correct class name for the binary Dice metric in recent versions
# The error "AttributeError: module 'torchmetrics' has no attribute 'Dice'"
# strongly suggests an issue with the torchmetrics installation or version.
# Re-running the pip install command and ensuring the kernel is refreshed
# is the primary fix. The line below is correct for recent torchmetrics versions.
metric_iou = torchmetrics.JaccardIndex(task="binary")
# The problematic line, now corrected:
metric_dice = DiceScore(num_classes=2) # Now imported directly from torchmetrics
metric_accuracy = torchmetrics.Accuracy(task="binary")

print("\nCalculating evaluation metrics...")

# Disable gradient calculation for inference
with torch.no_grad():
    # Iterate through the data loader
    # Ensure the loader is instantiated from a previous cell
    try:
        loader # Check if loader exists
    except NameError:
        print("DataLoader not found. Please run the cell that defines the dataset and loader.")
        # You might want to re-create the loader here if necessary
        # Example (assuming dataset and transforms are defined):
        # dataset = KvasirSegDataset(...) # Make sure KvasirSegDataset is defined/imported
        # image_transform = T.Compose([...]) # Make sure transforms are defined
        # mask_transform = T.Compose([...]) # Make sure transforms are defined
        # loader = DataLoader(dataset, batch_size=8, shuffle=True)
        # If you cannot re-create, you might need to stop execution
        exit() # Stop execution if loader is critical

    for i, (images, masks) in enumerate(loader):
        # Assuming loader outputs images and ground truth masks

        # Move data to the same device as the model (optional, but good practice if using GPU)
        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # images = images.to(device)
        # masks = masks.to(device)
        # model.to(device) # Ensure model is on the device
        # # Metrics also need to be on the same device
        # metric_iou.to(device)
        # metric_dice.to(device)
        # metric_accuracy.to(device)

        preds = model(images)

        # Apply sigmoid to predictions and threshold to get binary masks (0 or 1)
        # Ensure predictions and masks are on the same device and dtype for metric calculation
        predicted_masks_binary = (torch.sigmoid(preds) > 0.5).int() # Convert boolean to integer (0 or 1)

        # Update metrics with the current batch
        # Predicted masks should be int (0 or 1) and ground truth masks float or int
        # JaccardIndex and Dice expect inputs to be the same size
        # Ensure ground truth mask is also int (0 or 1) if needed by metric (ToTensor already makes it float)
        # Let's ensure the target is also binary int for clarity
        masks_binary = (masks > 0).int()

        metric_iou.update(predicted_masks_binary, masks_binary)
        metric_dice.update(predicted_masks_binary, masks_binary)
        metric_accuracy.update(predicted_masks_binary, masks_binary)

# Compute the final metric values
final_iou = metric_iou.compute()
final_dice = metric_dice.compute()
final_accuracy = metric_accuracy.compute()

# Print the metrics
print(f"Evaluation Metrics over the entire dataset:")
print(f"  Mean IoU: {final_iou.item():.4f}")
print(f"  Mean Dice: {final_dice.item():.4f}")
print(f"  Overall Accuracy: {final_accuracy.item():.4f}")

# Reset metrics for potential future use (optional)
metric_iou.reset()
metric_dice.reset()
metric_accuracy.reset()

Loaded model from /content/swinenet_kvasir_seg.pth

Calculating evaluation metrics...


/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: DiceScore metric currently defaults to `average=micro`, but will change to`average=macro` in the v1.9 release. If you've explicitly set this parameter, you can ignore this warning.
  warnings.warn(*args, **kwargs)



Original Feature map shapes from encoder:
  Stage 0: torch.Size([8, 56, 56, 128])
Permuting feature map 0 from NHWC torch.Size([8, 56, 56, 128]) to NCHW
  Stage 1: torch.Size([8, 28, 28, 256])
Permuting feature map 1 from NHWC torch.Size([8, 28, 28, 256]) to NCHW
  Stage 2: torch.Size([8, 14, 14, 512])
Permuting feature map 2 from NHWC torch.Size([8, 14, 14, 512]) to NCHW
  Stage 3: torch.Size([8, 7, 7, 1024])
Permuting feature map 3 from NHWC torch.Size([8, 7, 7, 1024]) to NCHW

Feature map shapes from encoder (after potential permute to NCHW):
  Stage 0: torch.Size([8, 128, 56, 56])
  Stage 1: torch.Size([8, 256, 28, 28])
  Stage 2: torch.Size([8, 512, 14, 14])
  Stage 3: torch.Size([8, 1024, 7, 7])

Checking channel match for MDCB inputs:
  MDCB 0: Expected 128, Actual 128
  MDCB 1: Expected 256, Actual 256
  MDCB 2: Expected 512, Actual 512
  MDCB 3: Expected 1024, Actual 1024

Shape before decoding: torch.Size([8, 1024, 7, 7])
Shape after decoder layer 0: torch.Size([8, 256, 14, 

In [ ]:
!zip -r /content/predicted_masks.zip /content/predicted_masks

from google.colab import files
files.download("/content/predicted_masks")

  adding: content/predicted_masks/ (stored 0%)
  adding: content/predicted_masks/predicted_mask_033.png (deflated 3%)
  adding: content/predicted_masks/predicted_mask_0383.png (deflated 2%)
  adding: content/predicted_masks/predicted_mask_0992.png (deflated 8%)
  adding: content/predicted_masks/predicted_mask_0335.png (deflated 0%)
  adding: content/predicted_masks/predicted_mask_0178.png (deflated 8%)
  adding: content/predicted_masks/predicted_mask_178.png (stored 0%)
  adding: content/predicted_masks/predicted_mask_0358.png (deflated 3%)
  adding: content/predicted_masks/predicted_mask_094.png (deflated 0%)
  adding: content/predicted_masks/predicted_mask_0389.png (deflated 6%)
  adding: content/predicted_masks/predicted_mask_247.png (stored 0%)
  adding: content/predicted_masks/predicted_mask_0828.png (deflated 4%)
  adding: content/predicted_masks/predicted_mask_0083.png (deflated 0%)
  adding: content/predicted_masks/predicted_mask_0370.png (deflated 1%)
  adding: content/predict

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>